In [22]:
from twelvedata import TDClient
import pprint
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import sqlite3
td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")

In [23]:
# Cryptocurrencies List, td.get_cryptocurrencies_list() see at: (https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)
all_Cryptos = td.get_cryptocurrencies_list().as_json()

#Print firtst item in list as visual aid
all_Cryptos[0]

{'symbol': '0xBTC/BTC',
 'available_exchanges': ['Hotbit', 'Mercatox'],
 'currency_base': '0xBitcoin',
 'currency_quote': 'Bitcoin'}

In [24]:
# Filter the symbols that are being compared to USD
usd_symbols = [crypto['symbol'] for crypto in all_Cryptos if '/USD' in crypto['symbol']]
print("Number of USD-paired symbols:", len(usd_symbols))

Number of USD-paired symbols: 1641


In [25]:
ts = td.time_series(
    symbol="BTC/USD",
    timezone="America/New_York",
    interval="30min",
    outputsize=5
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-24 20:00:00,29176.98,29180.66,29149.08,29164.83
2023-07-24 19:30:00,29185.18,29191.62,29168.57,29176.98
2023-07-24 19:00:00,29160.04,29206.97,29154.14,29185.33
2023-07-24 18:30:00,29154.74,29171.02,29138.01,29160.06
2023-07-24 18:00:00,29166.99,29191.93,29144.01,29154.74


In [26]:
ts = td.time_series(
    symbol="BTC/USD",
    interval="1month",
    start_date="2018-07-20",
    end_date="2023-07-20"
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-01,30466.73000,31862.21000,28850.00000,29164.83000
2023-06-01,27221.54000,31443.67000,24808.97000,30466.72000
2023-05-01,29240.49000,29850.00000,25810.00000,27223.08000
2023-04-01,28477.10938,31050.00000,26965.14000,29247.14000
2023-03-01,23135.34961,29168.71094,19582.88086,28477.10938
2023-02-01,23128.68945,25235.83984,21415.03906,23128.75977
2023-01-01,16528.02930,23944.60938,16492.97070,23127.67969
2022-12-01,17167.33008,18360.83984,16319.29980,16528.02930
2022-11-01,20492.08984,21470.58008,15500.33984,17167.33984


In [27]:
# Create a list to store the DataFrames for each symbol in the batch
dfs = []

# Loop through batches of symbols
symbol_batch = ['BTC/USD', #'ETH/USD', 
                'USDT/USD', 'XRP/USD', 'BNB/USD', 'ADA/USD', 'SOL/USD', 'DOGE/USD']

# Process each symbol in the current batch
for symbol in symbol_batch:
    try:
        ts = td.time_series(
            symbol=symbol,
            interval="1week",
            timezone="America/New_York",
            outputsize=208
        ).with_bbands(time_period=3).with_macd(fast_period=4, slow_period=8, signal_period=2).with_rsi(time_period=6).as_pandas()

        # Rename the 'close' column to the symbol name for clarity
        #ts.rename(columns={'close': symbol}, inplace=True)

        # Add a new numeric index column without dropping the original timestamp index
        ts.reset_index(inplace=True)

        # Add the DataFrame for the current symbol to the list
        dfs.append(ts)

    except Exception as e:
        print(f"Error fetching data for symbol '{symbol}': {str(e)}")

# Introduce a delay of 1 minute between API calls
time.sleep(60)

print("Total DataFrames fetched:", len(dfs))

Total DataFrames fetched: 7


In [28]:
dfs[1]

,datetime,open,high,low,close,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,rsi
0,2023-07-24,0.99985,1.00000,0.99962,0.99997,1.00017,0.99997,0.99976,-0.00006,-0.00006,0.00000,45.88096
1,2023-07-17,1.00010,1.00020,0.99941,0.99984,1.00034,1.00004,0.99974,-0.00007,-0.00007,-0.00000,41.41778
2,2023-07-10,1.00020,1.00057,0.99960,1.00009,1.00030,1.00007,0.99985,-0.00004,-0.00006,0.00002,47.72529
3,2023-07-03,0.99993,1.00050,0.99948,1.00020,1.00033,0.99996,0.99959,-0.00006,-0.00010,0.00003,50.54787
4,2023-06-26,0.99976,1.00057,0.99004,0.99993,1.00004,0.99987,0.99970,-0.00015,-0.00016,0.00001,43.74223
...,...,...,...,...,...,...,...,...,...,...,...,...
203,2019-09-02,1.00020,1.00380,0.99950,1.00180,1.00224,1.00090,0.99956,0.00042,0.00031,0.00010,57.28215
204,2019-08-26,1.00070,1.00300,0.99610,1.00020,1.00084,1.00043,1.00002,0.00014,0.00011,0.00003,51.59166
205,2019-08-19,1.00040,1.00360,0.99680,1.00070,1.00278,0.99933,0.99588,0.00013,0.00004,0.00009,53.44568
206,2019-08-12,0.99690,1.00500,0.99360,1.00040,1.00270,0.99930,0.99590,-0.00010,-0.00015,0.00005,52.59367


In [29]:
dfs[0]

,datetime,open,high,low,close,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,rsi
0,2023-07-24,30081.61000,30099.99000,28850.00000,29164.83000,30783.67391,29831.36000,28879.04609,342.99370,447.10209,-104.10839,52.57297
1,2023-07-17,30247.56000,30421.29000,29525.00000,30081.61000,30301.48581,30165.87667,30030.26752,616.69655,655.31888,-38.62233,63.75839
2,2023-07-10,30168.26000,31862.21000,29920.31000,30247.64000,30730.92442,30343.17667,29955.42891,736.48711,732.56355,3.92357,65.87354
3,2023-07-03,30613.57000,31525.10000,29715.87000,30168.38000,30793.78031,30420.71000,30047.63969,789.64840,724.71641,64.93199,65.41713
4,2023-06-26,30480.24000,31277.00000,29417.14000,30613.51000,33114.51179,29143.40333,25172.29488,783.35051,594.85242,188.49809,69.78504
...,...,...,...,...,...,...,...,...,...,...,...,...
194,2019-05-13,6979.75977,8385.21973,6621.29980,8196.58984,8990.69702,6962.84326,4934.98951,0.00000,0.00000,0.00000,0.00000
195,2019-05-06,5715.85986,7581.81982,5564.93994,6978.79980,7475.00030,5948.97998,4422.95966,0.00000,0.00000,0.00000,0.00000
196,2019-04-29,5157.91992,5846.87988,5072.00977,5713.14014,5862.11774,5388.59342,4915.06911,0.00000,0.00000,0.00000,0.00000
197,2019-04-22,5297.64014,5648.37988,4970.00000,5155.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [31]:
# Create a database connection using SQLAlchemy
engine = create_engine('sqlite:///crypto_data.db')

# Loop through the DataFrames in dfs and save each DataFrame to the database
for df in dfs:
    df.to_sql(name='crypto_data', con=engine, if_exists='replace', index=False)

# Close the database connection
#engine.dispose()

In [32]:
data = engine.execute("SELECT * FROM crypto_data")

for record in data:
    print(record)

AttributeError: 'Engine' object has no attribute 'execute'

In [ ]:
# Create a SQLite database and establish a connection
conn = sqlite3.connect('crypto_data.db')

# Loop through the DataFrames in dfs and save each DataFrame to the database
for df in dfs:
    df.to_sql(name='crypto_data', con=conn, if_exists='append', index=False)

# Close the database connection
conn.close()

In [30]:
def get_stock_data(symbol):
    try:
        td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")
        response = td.time_series(
            symbol=symbol[1],
            interval="30min",
            outputsize=50
            )
        
        if response:
            data = response.as_json()
            return data
        else:
            print("No data received from the API.")
            return None
            
    except Exception as e:
        print(f"Error occurred during the API call: {e}")
        return None

# Example usage for AAPL stock
symbol = "BTC/USD"
data = get_stock_data(symbol)

if data:
    pprint.pprint("API call successful!")
    pprint.pprint(data)

'API call successful!'
({'close': '14.65750',
  'datetime': '2023-07-20 15:30:00',
  'high': '14.66000',
  'low': '14.58000',
  'open': '14.59500',
  'volume': '10401770'},
 {'close': '14.59500',
  'datetime': '2023-07-20 15:00:00',
  'high': '14.60500',
  'low': '14.54000',
  'open': '14.54000',
  'volume': '4860082'},
 {'close': '14.54270',
  'datetime': '2023-07-20 14:30:00',
  'high': '14.59000',
  'low': '14.54000',
  'open': '14.56500',
  'volume': '2313557'},
 {'close': '14.56500',
  'datetime': '2023-07-20 14:00:00',
  'high': '14.61000',
  'low': '14.55000',
  'open': '14.61000',
  'volume': '2015613'},
 {'close': '14.61000',
  'datetime': '2023-07-20 13:30:00',
  'high': '14.65000',
  'low': '14.59000',
  'open': '14.65000',
  'volume': '2973305'},
 {'close': '14.64520',
  'datetime': '2023-07-20 13:00:00',
  'high': '14.65000',
  'low': '14.61000',
  'open': '14.62000',
  'volume': '2412057'},
 {'close': '14.62000',
  'datetime': '2023-07-20 12:30:00',
  'high': '14.64000',
